In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

入力パラメータ

In [2]:
end_step, save_step  = 7500,20

p1_r, p1_cx, p1_cy = 0.01, 0.10, 0.10
p2_r, p2_cx, p2_cy = 0.01, 0.10, 0.02

p_kn, p_rho = 1000000, 2650

dt = 0.0002

gravity_x, gravity_y = 0.0, -9.80665

p_cn = 393.0

変数の初期化

In [ ]:
p1_fx, p1_fy = 0.0, 0.0
p1_vx, p1_vy = 0.0, 0.0

p2_fx, p2_fy = 0.0, 0.0
p2_vx, p2_vy = 0.0, 0.0

p1_mass = np.pi*p_rho*p1_r*p1_r
p2_mass = np.pi*p_rho*p2_r*p2_r

dt_crit = 2.0*np.sqrt(p1_mass/p_kn)
print(dt_crit)

damping_constant = p_cn/(2.0*np.sqrt(p1_mass*p_kn))
print(damping_constant)

eb = 0.50
h = np.sqrt(np.log(eb)*np.log(eb) / (np.pi*np.pi + np.log(eb)*np.log(eb)))
c = 2.0*h*np.sqrt(p1_mass*p_kn)
print(h)
print(c)

（関数）接触力計算

In [4]:
def calculation_contactforce():

    global p1_fx, p1_fy, p2_fx, p2_fy

    dx = p1_cx - p2_cx
    dy = p1_cy - p2_cy
    d = np.sqrt(dx*dx + dy*dy)
    U = (p1_r + p2_r) - d

    if ( U > 0 ):
        f = p_kn * U
        nx = dx/d
        ny = dy/d

        rel_vx = p1_vx - p2_vx;
        rel_vy = p1_vy - p2_vy;

        p1_fx =  nx*f - rel_vx*p_cn
        p1_fy =  ny*f - rel_vy*p_cn
        p2_fx = -nx*f + rel_vx*p_cn
        p2_fy = -ny*f + rel_vy*p_cn
    else:
        p1_fx = 0.0
        p1_fy = 0.0
        p2_fx = 0.0
        p2_fy = 0.0

（関数）速度・位置の更新

In [5]:
def calculation_numericalintegration():

    global p1_vx, p1_vy, p1_cx, p1_cy

    p1_ax = (p1_fx/p1_mass) + gravity_x
    p1_ay = (p1_fy/p1_mass) + gravity_y

    p1_vx = p1_vx + p1_ax*dt
    p1_vy = p1_vy + p1_ay*dt

    p1_cx = p1_cx + p1_vx*dt
    p1_cy = p1_cy + p1_vy*dt

メイン

In [ ]:
plt_time = [0]
plt_p1_cx = [p1_cx]
plt_p1_cy = [p1_cy]

fig = plt.figure() #アニメーション用
ims = [] #アニメーション用

for i in range(1, end_step+1):
    calculation_contactforce()
    calculation_numericalintegration()

    if (i%save_step==0):
        plt_time.append(i*dt)
        plt_p1_cx.append(p1_cx)
        plt_p1_cy.append(p1_cy)

        im = plt.plot(p1_cx, p1_cy, marker='.', color = "green", markersize=40) #アニメーション用
        ims.append(im) #アニメーション用

anim = animation.ArtistAnimation(fig, ims, interval=50)

rc('animation', html='jshtml')
plt.close()
anim

グラフ作成（粒子の鉛直方向位置 vs 時間変化）

In [ ]:
plt.plot(plt_time, plt_p1_cy)
plt.ylabel('Particle position (m)')
plt.xlabel('Time (s)')
plt.show()

csvデータのダウンロード

In [8]:
csv = ""
plt_length = len(plt_time)

for i in range(plt_length):
    csv = csv + "{0},{1},{2}".format(plt_time[i], plt_p1_cx[i], plt_p1_cy[i]) + "\n"

open("data.csv", "wt").write(csv)

from google.colab import files
files.download("data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>